# Applicazione Python "Premi Nobel"

Attività scelta: 3. MongoDB Usecase e applicazione Python <br>
Progetto realizzato da Elena Curti (matricola: 146539 <mark>FORSE DA CAMBIARE?</mark>) con <mark> Python 3.11.0 (installato) oppure Python 3.9.13 (anaconda) ?

Ho usato i seguenti file JSON pubblici: <br>
    <ul> 
      <li> nobelPrizes.json: https://masterdataapi.nobelprize.org/2.1/nobelPrizes?offset=0&limit=664 </li>
      <li> laureates.json: https://masterdataapi.nobelprize.org/2.1/laureates?offset=0&limit=981 </li>
    </ul>
Essi contengono informazioni sui premi Nobel ed i relativi vincitori.

## Documentazione 
Documentazione dei dataset: https://app.swaggerhub.com/apis/NobelMedia/NobelMasterData/2.1

## File con i premi Nobel
Il file <i>JSON_Files/modificati/nobelPrizes2.json</i> contiene l'elenco dei premi Nobel, con la seguente struttura d'esempio:

```
{
    "awardYear": 2021,
    "category": "Economic Sciences",
    "categoryFullName": "The Sveriges Riksbank Prize in Economic Sciences in Memory of Alfred Nobel",
    "dateAwarded": "2021-10-11",
    "topMotivation": "for contributions ...",
    "prizeAmount": 10000000,
    "laureates": [
        {
            "id": "1007",
            "knownName": "David Card", // Solo per le persone
            "orgName": "Institute of International Law", // Solo per le organizzazioni
            "portion": "1/2",
            "sortOrder": "1",
            "motivation": "for his empirical contributions to labour economics"
        },
        { ... }
    ]
}
```

### Modifiche effettuate
Rispetto al file JSON originale (contenuto in <i>JSON_Files/originali/nobelPrizes.json</i>): 
<ul>
    <li> Ho rimosso i sottoalberi "/link" e "/meta" perchè contenenti informazioni non inerenti ai premi Nobel. <br></li>
    <li> 
        Ho rimosso la traduzione di alcuni campi. La struttura originale di tali campi era la seguente: 

```
"nome": {
    "en": "termine_inglese",
    "no": "termine_svedese",
    "se": "termine_norvegese"
}

```

<ul> 
<li style="list-style-type: none; ">
    Ho deciso di lasciare solo il termine inglese, trasformando quindi i campi in formato testuale. <br> 
    Ho eseguito questa modifica per i campi: "category", "categoryFullName", "laureates.knownName" e "laureates.motivation".
</li>
<li> Ho trasformato "awardYear" in un campo intero (originariamente testuale)</li>
<li> Ho rimosso i campi "laureates.fullName" e "laureates.nativeName" (il motivo è spiegato in seguito) </li>
</ul>

## File con i vincitori
Il file <i>JSON_Files/modificati/laureates2.json</i> contiene l'elenco dei vincitori dei premi Nobel, con la seguente struttura d'esempio:

```
// Per le persone
{
    "id": "102",
    "knownName": "Aage N. Bohr",
    "givenName": "Aage N.",
    "familyName": "Bohr",
    "fullName": "Aage Niels Bohr",
    "gender": "male",
    "birth": {
        "date": "1922-06-19",
        "place": {
            "cityNow": "Copenhagen",
            "countryNow": "Denmark"
        }
    },
    "death": {
        "date": "2009-09-08",
        "place": {
            "cityNow": "Copenhagen",
            "countryNow": "Denmark"
        }
    },
    "wikipedia": "https://en.wikipedia.org/wiki/Aage_Bohr"
} 

// Per le organizzazioni
{
    "id": "537",
    "orgName": "Amnesty International",
    "nativeName": "Amnesty International",
    "founded": {
        "date": "1961-00-00",
        "place": {
            "cityNow": "London",
            "countryNow": "United Kingdom"
        }
    },
    "wikipedia": "https://en.wikipedia.org/wiki/Amnesty_International"
}
```

### Modifiche effettuate
Rispetto al file JSON originale (contenuto in <i>JSON_Files/originali/laureates.json</i>): 
<ul>
    <li>  
        Ho rimosso i sottoalberi "/link" e "/meta" perchè contenenti informazioni non inerenti ai vincitori.
    </li>
    <li> 
        Come prima, ho rimosso la traduzione svedese e norvegese dei seguenti campi (lasciando solo quella inglese): "knownName", "givenName", "givenName", "fullName", "cityNow", "countryNow", "orgName"
    </li>
    <li>
        Ho modificato i campi "birth.place", "death.place" e "founded.place" rimuovendo i sotto-campi "city", "country", "continent" e "locationString". In questo modo ho semplificato la struttura del file, rimuovendo informazioni molto specifiche sui luoghi geografici e non inerenti al vincitore del premio.
    </li>
    <li> Ho rimosso il campo "nobelPrizes", contenente l'elenco dei premi Nobel vinti dalla persona/organizzazione, perchè già memorizzati nel file nobelPrizes2.json.
</ul>

## Struttura delle Collection
Nelle interrogazioni proposte nell'applicazione, ho dato la possibilità all'utente di scegliere se visionare i vincitori dei premi in modo sintetico (solo il nome) o dettagliato (tutti i campi). Quindi, ho deciso di non unire i file laureates2.json e nobelPrizes2.json, ma di tenerli distinti. Ho deciso, inoltre, di lasciare in nobelPrizes2.json solo i campi "laureates.knownName" e "laureates.orgName", rimuovendo quindi fullName e nativeName. <br>
<mark> DA FARE </mark>
ho inserito i seguenti indici perche ... <br>

## Applicazione
Sono previsti due modi per eseguire l'applicazione:


### Operazioni iniziali
Eseguire il codice nelle celle, seguendo l'ordine proposto: <br>
Inserire nella seguente variabile il path della cartella con i file JSON:

In [80]:
path_cartella_file_json = 'C:\\Users\\utente\\Desktop\\JSON_Nobel\\'

<mark> CAMBIA I NOMI DEI FILE DELLE COSTATI

Il seguente codice crea ed inizializza un database chiamato <i>premi_nobel</i>, formato dalle due collections <i>laureates2</i> e <i>nobelPrizes2</i>, contenenti i dati memorizzati nei relativi file JSON. <br>
<b>Eseguedo questo codice, gli eventuali dati memorizzati nelle collections e non presenti nei file JSON saranno eliminati. </b>

In [81]:
import json, sys
from pymongo import MongoClient

def carica_dati(nomeFile, collection):
    ''' Funzione che carica il file json nella collection. '''
    collection.drop()
    try:
        with open(path_cartella_file_json+nomeFile, encoding='utf8') as file:
            file_data = json.load(file)
    except FileNotFoundError:
        print("Path inserito non valido!", file=sys.stderr)
        return False
    collection.insert_many(file_data) 
    return True

# Costanti
FILE_LAUREATES = "desktop_laureates2.json"
FILE_NOBEL_PRICES = "desktop_nobelPrizes2.json"

# Inizializzazione del database
myclient = MongoClient("mongodb://localhost:27017/")
db = myclient["premi_nobel"]
collection_laureates2 = db["laureates2"]
collection_nobelPrices2 = db["nobelPrizes2"]

if (carica_dati(FILE_LAUREATES, collection_laureates2) and carica_dati(FILE_NOBEL_PRICES, collection_nobelPrices2)):
    print("Database inizializzato correttamente. Eseguire l'applicazione...")

Database inizializzato correttamente. Eseguire l'applicazione...


In [84]:
def print_nobel(nobel):
    print("Anno: " + str(nobel.get('awardYear', "-")))
    print("Categoria: " + str(nobel.get('category', "-")) + ", " + str(nobel.get('categoryFullName', "")))
    print("Data: " + str(nobel.get('dateAwarded', "-")))
    print("Motivazione: " + str(nobel.get('topMotivation', "-")))
    print("Premio: " + str(nobel.get('prizeAmountAdjusted', "-")))



### Interrogazioni
#### 1. Cercare tutti i vincitori dei premi Nobel tra una categoria

In [85]:
print("Cercare tutti i vincitori dei premi Nobel di una categoria")

print("Quale categoria? ")
lista_categorie = collection_nobelPrices2.distinct("category")
i = 0
for cat in lista_categorie:
    print(str(i)+ " -- " + cat)
    i+=1
# num_categoria = int(input(">> "))
num_categoria = 0
try:
    lista = list(collection_nobelPrices2.find({"category":lista_categorie[num_categoria]})) #, {"laureates.knownName":1, "laureates.orgName":1, "_id":1})))
    for x in lista:
        print("*************")
        print_nobel(x)
except IndexError:
    print("Numero inserito non valido!")

Cercare tutti i vincitori dei premi Nobel di una categoria
Quale categoria? 
0 -- Chemistry
1 -- Economic Sciences
2 -- Literature
3 -- Peace
4 -- Physics
5 -- Physiology or Medicine
*************
Anno: 1901
Categoria: Chemistry, The Nobel Prize in Chemistry
Data: -
Motivazione: -
Premio: -
*************
Anno: 1902
Categoria: Chemistry, The Nobel Prize in Chemistry
Data: -
Motivazione: -
Premio: -
*************
Anno: 1903
Categoria: Chemistry, The Nobel Prize in Chemistry
Data: -
Motivazione: -
Premio: -
*************
Anno: 1904
Categoria: Chemistry, The Nobel Prize in Chemistry
Data: -
Motivazione: -
Premio: -
*************
Anno: 1905
Categoria: Chemistry, The Nobel Prize in Chemistry
Data: -
Motivazione: -
Premio: -
*************
Anno: 1906
Categoria: Chemistry, The Nobel Prize in Chemistry
Data: -
Motivazione: -
Premio: -
*************
Anno: 1907
Categoria: Chemistry, The Nobel Prize in Chemistry
Data: -
Motivazione: -
Premio: -
*************
Anno: 1908
Categoria: Chemistry, The Nob

In [ ]:
from pprint import pprint

continuare = True
while(continuare):
    print ('********************')
    print ('1 -- Option 1' )
    print ('2 -- Option 2' )
    print ('3 -- Option 3' )
    print ('0 -- Exit' )
    option = input('>> ')

    #Check what choice was entered and act accordingly
    if option == "0":
        print("Applicazione terminata")
        continuare = False
    elif option == "1":
        
        print("Hai scelto: 1")
        pprint(list(collection_laureates2.find({}).limit(3)))

        #pprint(db_laureates2.find_one({"knownName":"Alfonso García Robles"}))

    elif option == "2":
        print("Hai scelto: 2")
    elif option == "Hai scelto: 3":
        print("3")
    else:
        print('Invalid option. Please enter a number between 1 and 4.')